In [1]:
from enum import Enum, unique
import toml
import numpy as np
from rich.tree import Tree
import pandas as pd

import sys
sys.path.append('../src')
from sysloss.element import *

In [70]:
import rustworkx as rx
import numpy as np
from rich.tree import Tree
import json
import pandas as pd

#def df_style(val):
#    return "font-weight: bold"

class System:
    """System to be analyzed.
    """
    
    def __init__(self, name, source):
        self.g = None
        if type(name) != str:
            raise ValueError("Error: Name \"{}\" of system must be a string!".format(name))
        elif not isinstance(source, Source):
            raise ValueError("Error: First element of system must be a source!")
        else:
            self.g = rx.PyDAG(check_cycle=True, multigraph=False, attrs={})
            pidx = self.g.add_node(source)
            self.g.attrs[source.params['name']] = pidx
            
    def __get_index(self, name):
        """Get node index from element name"""
        if name in self.g.attrs:
            return self.g.attrs[name]

        return -1
    
    def __chk_parent(self, parent):
        """Check if parent exists"""
        if type(parent) != str:
            raise ValueError("Error: Parent name must be a string!")
        # check if parent exists
        if not parent in self.g.attrs.keys():
            raise ValueError("Error: Parent name \"{}\" not found!".format(parent))
            return False
        
        return True
    
    # check if element name is valid
    def __chk_name(self, name):
        """Check if element name is valid"""
        # check name type
        if type(name) != str:
            raise ValueError("Error: Element name must be a string!")
            return False
        # check if exists exists
        pidx = self.__get_index(name)
        if name in self.g.attrs.keys():
            raise ValueError("Error: Element name \"{}\" is already used!".format(name))
            return False
        
        return True
                  
    def __get_childs(self, rev=True):
        """Get dict of parent/childs"""
        if rev == True:
            childs = list(reversed(rx.bfs_successors(self.g, 0)))
        else:
            childs = list(rx.bfs_successors(self.g, 0))
        cdict = {}
        for c in childs:
            cs = []
            for l in c[1]:
                cs += [self.g.attrs[l.params['name']]]
            cdict[self.g.attrs[c[0].params['name']]] = cs
        return cdict
    
    def __get_nodes(self):
        """Get list of nodes in system"""
        nodes = []
        for n in self.g.nodes():
            nodes += [self.g.attrs[n.params['name']]]
        return sorted(nodes)
    
    def __get_parents(self):
        """Get list of parent of each child"""
        nodes = self.__get_nodes()
        ps = np.zeros(max(nodes)+1, dtype=np.int32)
        for n in nodes:
            if n != 0:
                pname = [i for i in self.g.attrs if self.g.attrs[i]==n]
                ps[n] = case1.g.attrs[self.g.predecessors(self.g.attrs[pname[0]])[0].params['name']]
        return list(ps)
    
    def __get_edges(self):
        """Get list of element connections (edges)"""
        return list(reversed(rx.dfs_edges(self.g, 0)))
    
    def __get_leaves(self):
        """Get list of leaf nodes"""
        nodes = self.__get_nodes()
        ls = np.zeros(max(nodes)+1, dtype=np.int32)
        for n in nodes:
            if self.g.out_degree(n) == 0:
                ls[n] = 1
        return list(ls)
        
    def __sys_vars(self):
        """Get system variable lists"""
        vn = max(self.__get_nodes())+1 # highest node index + 1
        v = list(np.zeros(vn)) # voltages
        i = list(np.zeros(vn)) # currents
        return v, i 

    def __make_rtree(self, adj, node):
        """Create Rich tree"""
        tree = Tree(node)
        for child in adj.get(node, []):
            tree.add(self.__make_rtree(adj, child))
        return tree
    
    def add_element(self, parent:str, *, element):
        """Add element to system"""
        # check that parent exists
        if not self.__chk_parent(parent):
            raise ValueError("Error: Parent name does not exist!")
            return
        
        # check that element name is unique
        if not self.__chk_name(element.params['name']):
            raise ValueError("Error: Element name already taken!")
            return
        
        pidx = self.__get_index(parent)
        
        # check that parent allows element type as child
        if not element.element_type in self.g[pidx].child_types:
            raise ValueError("Error: Parent does not allow child of type {}!".format(element.element_type.name))
            return
        
        cidx = self.g.add_child(pidx, element, None)
        self.g.attrs[element.params['name']] = cidx
        
    def change_element(self, *, name:str, element):
        """Replace element with a new one"""
        # if element name changes, check that it is unique
        if name != element.params['name']:
            if not self.__chk_name(element.params['name']):
                raise ValueError("Error: Element name already taken!")
                return
            
        eidx = self.__get_index(name)
        # check that parent allows element type as child
        if eidx != 0:
            parents = self.__get_parents()
            if not element.element_type in self.g[parents[eidx]].child_types:
                raise ValueError("Error: Parent does not allow child of type {}!".format(element.element_type.name))
                return

        self.g[eidx] = element
        # replace node name in graph dict
        del[self.g.attrs[name]]
        self.g.attrs[element.params['name']] = eidx
        
    def del_element(self, *, name:str, del_childs: bool=True):
        eidx = self.__get_index(name)
        if eidx == -1:
            raise ValueError("Error: Element name does not exist!")
        if eidx == 0:
            raise ValueError("Error: Cannot delete source node!")
        parents = self.__get_parents()
        leaves = self.__get_leaves()
        childs = self.__get_childs()
        # if not leaf, check if child type is allowed by parent type
        if leaves[eidx] == 0:
            #print(childs[eidx], parents[eidx])
            for c in childs[eidx]:
                if not self.g[c].element_type in self.g[parents[eidx]].child_types:
                    raise ValueError("Error: Parent and child of element are not compatible!")
        # delete childs first if selected
        if del_childs:
            for c in rx.descendants(self.g, eidx):
                print(c, eidx)
                del[self.g.attrs[self.g[c].params['name']]]     
                self.g.remove_node(c)
        # delete node
        self.g.remove_node(eidx)
        del[self.g.attrs[name]]
        # restore links between new parent and childs, unless deleted
        if not del_childs:                     
            if leaves[eidx] == 0:
                for c in childs[eidx]:
                    self.g.add_edge(parents[eidx], c, None)
               
    def tree(self, name=''):
        """Print tree structure, starting from node name"""
        if not name == '':
            if not name in self.g.attrs.keys():
                raise ValueError("Error: Element name is not valid!")
            root = name
        else:
            root = self.g[0].params['name']
            
        adj = rx.bfs_successors(self.g, self.g.attrs[root])
        ndict = {}
        for i in adj:
            c = []
            for j in i[1]:
                c += [j.params['name']]
            ndict[i[0].params['name']] = c
        return self.__make_rtree(ndict, root)
        
    def __sys_init(self):
        """Create vectors of init values for solver"""
        v, i = self.__sys_vars()
        for n in self.__get_nodes():
            v[n] = self.g[n]._get_outp_voltage()
            i[n] = self.g[n]._get_inp_current()
        return v, i
    
    def __fwd_prop(self, v:float, i:float):
        """Forward propagation of voltages"""
        vo, _ = self.__sys_vars()
        # update output voltages (per node)
        for n in self.__nodes:
            if self.__leaves[n] == 1: # leaf
                if n == 0: # root             
                    vo[n] = self.g[n]._solv_outp_volt(.0, .0, .0)
                    #print('leaf, root:', vo[n], n)
                else:
                    p = self.__parents[n]
                    vo[n] = self.g[n]._solv_outp_volt(v[p], i[p], .0)
                    #print('leaf: ', vo[n], n)
            else:
                # add currents into childs
                isum = 0
                for c in self.__childs_f[n]: 
                    isum += i[c]
                if n == 0: # root
                    vo[n] = self.g[n]._solv_outp_volt(.0, .0, isum)
                    #print('root:', vo[n], n)
                else:
                    p = self.__parents[n]
                    vo[n] = self.g[n]._solv_outp_volt(v[p], i[p], isum)
                    #print('element:' , vo[n], n)
        
        return vo

    def __back_prop(self, v:float, i:float):
        """Backward propagation of currents"""
        _, io = self.__sys_vars()
        # update input currents (per edge)
        for e in self.__edges:
            if self.__leaves[e[1]] == 1: # leaf
                io[e[1]] = self.g[e[1]]._solv_inp_curr(v[e[0]], .0, .0)
            else:
                c = self.__childs_b[e[1]]
                io[e[1]] = self.g[e[1]]._solv_inp_curr(v[e[0]], v[e[1]], i[c[0]])
        # add currents into childs from root
        if self.__childs_b != {}:
            for c in self.__childs_b[0]:
                io[0] += i[c]
        
        return io
    
    def __rel_update(self):
        """Update lists with element relationships"""
        self.__nodes = self.__get_nodes()
        self.__edges = self.__get_edges()
        self.__childs_f = self.__get_childs(rev = False)
        self.__childs_b = self.__get_childs(rev = True)
        self.__parents = self.__get_parents()
        self.__leaves = self.__get_leaves()
        
    def __get_parent_name(self, node):
        if node == 0:
            return ''
        
        return self.g[self.__parents[node]].params['name']
        
    
    def solve(self, *, vtol=1e-5, itol=1e-6, maxiter=1000, quiet=True):
        """Analyze system"""
        self.__rel_update()
        # initial condition
        v, i = self.__sys_init()
        # solve system function
        iters = 1
        while iters <= maxiter:
            vi = self.__fwd_prop(v, i)
            ii = self.__back_prop(vi, i)
            if np.allclose(np.array(v), np.array(vi), rtol=vtol) and np.allclose(np.array(i), np.array(ii), rtol=itol):
                if not quiet:
                    print('Tolerances met after {} iterations'.format(iters))
                break;
            v, i = vi, ii
            iters += 1
            
        if iters > maxiter:
            print('Analysis aborted after {} iterations'.format(iters - 1))
            return None
            
        # calculate results for each node       
        names, parent, typ, pwr, loss, eff, warn, vsi, iso = [], [], [], [], [], [], [], [], []
        for n in self.__nodes: # [vi, vo, ii, io]
            names += [self.g[n].params['name']]
            vi = v[n]
            vo = v[n]
            ii = i[n]
            io = i[n]
            
            if n == 0: # root 
                #parent += ['']
                #print(n, v[n], v[n], i[n], i[n], p, l)
                vi = v[n] + self.g[n].params['rs'] * ii
            elif self.__leaves[n] == 1: # leaf
                vi = v[self.__parents[n]]
                io = 0.0
                #parent += [self.g[self.__parents[n]].params['name']]
                #print(n, v[n], 0.0, i[n], 0.0, p, l)
            else:
                io = 0.0
                for c in self.__childs_f[n]: 
                    io += i[c]
                vi = v[self.__parents[n]]
                #parent += [self.g[self.__parents[n]].params['name']]
                #print(n, v[self.__parents[n]], v[n], i[n], isum, p, l)
                
            parent += [self.__get_parent_name(n)]
            p, l, e = self.g[n]._solv_pwr_loss(vi, vo, ii, io)    
            pwr += [p]
            loss += [l]
            eff += [e]
            typ += [self.g[n].element_type.name]
            warn += [self.g[n]._solv_get_warns(vi, vo, ii, io)]
            vsi += [vi]
            iso += [io]
            
        # remove unused node indices
        vso, isi = [], []
        for n in range(len(v)):
            if n in self.__nodes:
                vso += [v[n]]
                isi += [i[n]]
            
        # report
        res = {}
        res['Element'] = names
        res['Type'] = typ
        res['Parent'] = parent
        res['Vin (V)'] = vsi
        res['Vout (V)'] = vso
        res['Iin (A)'] = isi
        res['Iout (A)'] = iso
        res['Power (W)'] = pwr #[res['Power (W)'].sum() - res['Power (W)'][0]]
        res['Loss (W)'] = loss #[res['Loss (W)'].sum()]
        res['Efficiency (%)'] = eff
        res['Warnings'] = warn
        df = pd.DataFrame(res)
        tpwr = abs(vsi[0] * i[0])
        tloss = df['Loss (W)'].sum()
        w = 'None'
        if sum(df['Warnings'] != '') > 0:
            w = 'Yes'
        eff = 0.0
        if tpwr > 0.0:
            eff = (tpwr - tloss)/tpwr
        df.loc[len(df)] = ['System total', '', '', vsi[0], 0.0, i[0], 0.0, tpwr, tloss, eff, w]
        
        return df
    
    def params(self, limits=False):
        """Return element parameters"""
        self.__rel_update()
        # extract params
        
        names, typ, parent, vo, vdrop, iq, rs, eff, ii, pwr = [], [], [], [], [], [], [], [], [], []
        lii, lio, lvi, lvo = [], [], [], []
        for n in self.__nodes:
            names += [self.g[n].params['name']]
            typ += [self.g[n].element_type.name]
            _vo, _vdrop, _iq, _rs, _eff, _ii, _pwr = '', '', '', '', '', '', ''
            if self.g[n].element_type == ElementTypes.SOURCE:
                _vo = self.g[n].params['vo']
                _rs = self.g[n].params['rs']
            elif self.g[n].element_type == ElementTypes.LOAD:
                if 'pwr' in self.g[n].params:
                    _pwr = self.g[n].params['pwr']
                else:
                    _ii =  self.g[n].params['ii']
            elif self.g[n].element_type == ElementTypes.CONVERTER:
                _vo = self.g[n].params['vo']
                _iq = self.g[n].params['iq']
                _eff = self.g[n].params['eff']
            elif self.g[n].element_type == ElementTypes.LINREG:
                _vo = self.g[n].params['vo']
                _vdrop = self.g[n].params['vdrop']
                _iq = self.g[n].params['iq']
            elif self.g[n].element_type == ElementTypes.LOSS:
                _vdrop = self.g[n].params['vdrop']
                _rs = self.g[n].params['rs']
            vo += [_vo]
            vdrop += [_vdrop]
            iq += [_iq]
            rs += [_rs]
            eff += [_eff]
            ii += [_ii]
            pwr += [_pwr]
            parent += [self.__get_parent_name(n)]
            if limits:
                lii += [self.g[n].limits['ii']]
                lio += [self.g[n].limits['io']]
                lvi += [self.g[n].limits['vi']]
                lvo += [self.g[n].limits['vo']]
        # report
        res = {}
        res['Element'] = names
        res['Type'] = typ
        res['Parent'] = parent
        res['vo (V)'] = vo
        res['vdrop (V)'] = vdrop
        res['iq (A)'] = iq
        res['rs (Ohm)'] = rs
        res['eff (%)'] = eff
        res['ii (A)'] = ii
        res['pwr (W)'] = pwr
        if limits:
            res['vi limits (V)'] = lvi
            res['vo limits (V)'] = lvo
            res['ii limits (A)'] = lii
            res['io limits (A)'] = lio
        return pd.DataFrame(res)
    
    def save(self, fname):
        """Save system as json file"""
        self.__rel_update()
        

![case1](case_1.svg)

In [56]:
case1 = System('Bluetooth sensor', Source('3V coin', vo=3, rs=13e-3))
case1.add_element('3V coin', element=Converter('1.8V buck', vo=-1.8, eff=0.87, iq=12e-6))
#case1.add_element('1.8V buck', element=Loss('Resistor2', rs=5.0))
case1.add_element('1.8V buck', element=PLoad('MCU', pwr=27e-3))
case1.add_element('3V coin', element=Converter('5V boost', vo=5, eff=0.91, iq=42e-6))
case1.add_element('5V boost', element=ILoad('Sensor', ii=15e-3))
case1.add_element(parent='5V boost', element=Loss('RC filter', rs=33.0))
case1.add_element('RC filter', element=LinReg('LDO 2.5V', vo=2.5, vdrop=0.27, iq=150e-6))
case1.add_element('LDO 2.5V', element=PLoad('ADC', pwr=15e-3))
#case1.g.attrs
case1.tree()

3V coin
├── 5V boost
│   ├── RC filter
│   │   └── LDO 2.5V
│   │       └── ADC
│   └── Sensor
└── 1.8V buck
    └── MCU

In [57]:
df = case1.solve(quiet=False)
df

Tolerances met after 8 iterations


,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.000000,2.999718,0.021665,0.021665,0.064988,0.000006,0.999906,
1,1.8V buck,CONVERTER,3V coin,2.999718,-1.800000,0.010358,0.015000,0.000000,0.004070,0.868992,
2,MCU,LOAD,1.8V buck,-1.800000,0.000000,0.015000,0.000000,0.027000,0.000000,100.000000,
3,5V boost,CONVERTER,3V coin,2.999718,5.000000,0.011307,0.021150,0.000000,0.003167,0.906620,
4,Sensor,LOAD,5V boost,5.000000,0.000000,0.015000,0.000000,0.075000,0.000000,100.000000,
5,RC filter,LOSS,5V boost,5.000000,4.797050,0.006150,0.006150,0.000000,0.001248,0.959410,
6,LDO 2.5V,LINREG,RC filter,4.797050,2.500000,0.006150,0.006000,0.000000,0.014502,0.508443,
7,ADC,LOAD,LDO 2.5V,2.500000,0.000000,0.006000,0.000000,0.015000,0.000000,100.000000,
8,System total,,,3.000000,0.000000,0.021665,0.000000,0.064994,0.022994,0.646216,None


In [58]:
case1.params()

,Element,Type,Parent,vo (V),vdrop (V),iq (A),rs (Ohm),eff (%),ii (A),pwr (W)
0,3V coin,SOURCE,,3,,,0.013,,,
1,1.8V buck,CONVERTER,3V coin,-1.8,,0.000012,,0.87,,
2,MCU,LOAD,1.8V buck,,,,,,,0.027
3,5V boost,CONVERTER,3V coin,5,,0.000042,,0.91,,
4,Sensor,LOAD,5V boost,,,,,,0.015,
5,RC filter,LOSS,5V boost,,0.0,,33.0,,,
6,LDO 2.5V,LINREG,RC filter,2.5,0.27,0.00015,,,,
7,ADC,LOAD,LDO 2.5V,,,,,,,0.015


In [10]:
#case1.change_element(name='MCU', element=Load('Big MCU', pwr=130e-3))
#case1.change_element(name='MCU', element=Loss('Resistor1', res=133.0))
#case1.change_element(name='1.8V buck', element=Loss('Resistor2', res=7.8))
#case1.add_element('ADC', element='dummy')
case1.del_element(name='5V boost', del_childs=False)
case1.tree()

3V coin
├── Sensor
├── RC filter
│   └── LDO 2.5V
│       └── ADC
└── 1.8V buck
    └── MCU

In [11]:
case1.solve()

,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.00000,2.99959,0.031508,0.031508,0.094512,0.000013,0.999863,
1,1.8V buck,CONVERTER,3V coin,2.99959,-1.80000,0.010358,0.015000,0.000000,0.004070,0.868992,
2,MCU,LOAD,1.8V buck,-1.80000,0.00000,0.015000,0.000000,0.027000,0.000000,100.000000,
3,Sensor,LOAD,3V coin,2.99959,0.00000,0.015000,0.000000,0.044994,0.000000,100.000000,
4,RC filter,LOSS,3V coin,2.99959,2.79664,0.006150,0.006150,0.000000,0.001248,0.932341,
5,LDO 2.5V,LINREG,RC filter,2.79664,2.50000,0.006150,0.006000,0.000000,0.002199,0.872127,
6,ADC,LOAD,LDO 2.5V,2.50000,0.00000,0.006000,0.000000,0.015000,0.000000,100.000000,
7,System total,,,3.00000,0.00000,0.031508,0.000000,0.094525,0.007531,0.920329,None


In [ ]:
par = {'source': {'vo':3.0, 'rs':0.007}, 'limits':LIMITS_DEFAULT}
toml_string = toml.dumps(par)  # Output to a string
with open('source.toml', "w") as f:
    toml.dump(par, f)

In [ ]:
s=Source.from_file('battery 3V', fname='source.toml')
s.params, s.limits

In [ ]:
with open('source.toml') as toml_file:
    toml_dict = toml.load(toml_file)
toml_dict

| Element | vo (V)   | vdrop (V) | iq (A) | rs (Ohm) | eff (%) | ii (A) | pwr (W) |
|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|
|  **Source**  | # |     |   | (#) |  |  |  |
|  **ILoad**   |     |     || || #||
|  **PLoad**   |     |    ||||| #|
|  **Converter**   |  #   |     | (#)||#|||
|  **LinReg**  |  # | (#)   | (#) |||||
|  **Loss**  |     | #   || # ||||

In [76]:
case2 = System('Case2', Source('3V coin', vo=3, rs=13e-3))
case2.add_element('3V coin', element=Loss('RC filter', rs=33.0))
case2.solve(quiet=False)

Tolerances met after 2 iterations


,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.0,3.0,0.0,0.0,0.0,0.0,100.0,
1,RC filter,LOSS,3V coin,3.0,3.0,0.0,0.0,0.0,0.0,0.0,
2,System total,,,3.0,0.0,0.0,0.0,0.0,0.0,0.0,None


In [67]:
a = {  }
a == {}

True